## Importing required packages

In [4]:
import json
import csv
import pandas as pd

## Cleaning raw JSON data from Google Location History and storing it into CSV file

In [3]:
data = json.load(open('Location History.json'))
with open('GoogleLH.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    heading = ["timestampMs", "latitude", "longitude", "accuracy"]
    wr.writerow(heading)

with open('GoogleLH.csv', 'a') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for locs in data["locations"]:
        valList = [None] * 4
        for key, value in locs.iteritems():
            if key == "timestampMs":
                valList[0] = value
            elif key == "latitudeE7":
                valList[1] = (value*1.0)/(10**7)
            elif key == "longitudeE7":
                valList[2] = (value*1.0)/(10**7)
            elif key == "accuracy":
                valList[3] = value
        wr.writerow(valList)